<a href="https://colab.research.google.com/github/gitsunder237/CC_de_Git_Akuma/blob/master/Tensorflow_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from   sklearn.preprocessing import LabelEncoder
from   sklearn.preprocessing import OneHotEncoder
from   sklearn.preprocessing import StandardScaler
from   sklearn.model_selection import train_test_split
from   sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder,  LabelBinarizer, OneHotEncoder
from sklearn.preprocessing import StandardScaler , MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.impute import SimpleImputer, KNNImputer

In [14]:
raw_data = pd.read_csv("Churn_Modelling.csv")


In [6]:
raw_data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data_quant = raw_data.drop(["RowNumber","Surname","CustomerId"], axis = 1)
data_quant.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [24]:
# Splitting the dataset into the Training set and Test set
X = raw_data.iloc[:, 3:-1]  # Remove .values to keep it as a DataFrame
y = raw_data.iloc[:, -1].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [25]:
# Construire un pipeline pour réaliser le pré-traitement des données
# conformément aux résultats des précédentes questions
numerical_features = make_column_selector(dtype_include = np.number)
nominal_features  = make_column_selector(dtype_exclude = np.number)

nominal_pipeline = make_pipeline(SimpleImputer(strategy = "most_frequent"),
                                OneHotEncoder(drop = "first")
                                )

numerical_pipeline = make_pipeline(KNNImputer(n_neighbors = 5),
                                  RobustScaler()
                                  )
preprocessor = make_column_transformer((nominal_pipeline, nominal_features),
                                       (numerical_pipeline, numerical_features)
                                       )
X_train = preprocessor.fit_transform(X_train)
X_test  = preprocessor.transform(X_test)
X_train.shape

(8000, 11)

In [26]:
# Standardisation des variables

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [27]:
# II - Construction du réseau de neurone

# Initialisation du réseau de neurone
ann = tf.keras.models.Sequential()

# Ajout des variables explicatives et de la première couche
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Ajout de la deuxième couche
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Ajout de la couche de sortie
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))



In [28]:
# Entraînement du modèle

# Compilation du réseau de neurone artificielle
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [29]:
# Entrainement du modèle proprement dit
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.5609 - accuracy: 0.7153
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4575 - accuracy: 0.7983
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4414 - accuracy: 0.8014
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4323 - accuracy: 0.8059
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4261 - accuracy: 0.8102
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4209 - accuracy: 0.8126
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4156 - accuracy: 0.8159
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4100 - accuracy: 0.8177
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4017 - accuracy: 0.8260
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3934 - accura

In [30]:
# Prédictions sur les données de test

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(-1,1), y_test.reshape(-1,1)),1))



63/63 [==============================] - 1s 4ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [31]:
# Making the Confusion Matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[1504   91]
 [ 193  212]]


0.858